<a href="https://colab.research.google.com/github/kjaewon4/ArtPlanner/blob/main/ml-server/notebooks/sd-style-transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diffusers transformers ftfy accelerate
!pip install python-multipart
!pip install --upgrade transformers==4.44.1
import transformers
print(transformers.__version__)  # 4.44.1 이 출력되어야 합니다.
!pip install --upgrade --no-deps fastapi uvicorn nest-asyncio pyngrok
!pip install fastapi uvicorn starlette nest-asyncio pyngrok

In [ ]:
from diffusers import StableDiffusionImg2ImgPipeline
import torch
from PIL import Image

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    torch_dtype=torch.float32
).to("cuda")

print("Stable Diffusion 파이프라인 정상 로드됨")

In [ ]:
# 3. FastAPI 서버 정의
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import FileResponse
import os, shutil
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

# CORS 설정 추가
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # 또는 ["http://localhost:3000"]으로 제한할 수 있음
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

os.makedirs("/content/uploads", exist_ok=True)
os.makedirs("/content/outputs", exist_ok=True)

@app.post("/api/sd-style-transfer")
async def sd_style_transfer(
    content: UploadFile    = File(...),         # ① File을 제일 먼저
    prompt: str            = Form(...),         # ② 그 다음 Form
    strength: float        = Form(0.7),         # ③ 숫자도 그대로 float
    guidance_scale: float  = Form(7.5)          # ④
):
    try:
        input_path  = f"/content/uploads/{content.filename}"
        output_path = f"/content/outputs/styled_{content.filename}"

        # 파일 저장
        with open(input_path, "wb") as f:
            shutil.copyfileobj(content.file, f)

        # 변환
        image = Image.open(input_path).convert("RGB").resize((512, 512))
        result = pipe(
            prompt=prompt,
            image=image,
            strength=strength,
            guidance_scale=guidance_scale
        ).images[0]
        result.save(output_path)
        return FileResponse(output_path, media_type="image/png")

    except Exception as e:
        return JSONResponse(status_code=400, content={"error": str(e)})


In [ ]:
# 4. ngrok으로 외부 공개
from pyngrok import ngrok, conf
import nest_asyncio
import uvicorn

# 1) 발급받은 토큰을 설정
conf.get_default().auth_token = "YOUR_NGROK_TOKEN"

# 2) 이제 터널 생성
ngrok_tunnel = ngrok.connect(8000)
print("Public URL:", ngrok_tunnel.public_url)

# FastAPI 서버 실행
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)